In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_data() -> pd.DataFrame:
    l = pd.read_csv('labelling-upper-bounds.csv')

    l['VALID_UB'] = l.algorithm_completed
    l['BEST_POSSIBLE_UB'] = l.algorithm_completed
    l['UB_VALUE'] = l.obj

    m = pd.read_csv('model-based-upper-bounds.csv')

    fw = m[m.algorithm.str.contains('frankwolfe')].copy()
    im = m[~m.algorithm.str.contains('frankwolfe')].copy()

    fw['VALID_UB'] = True
    fw['BEST_POSSIBLE_UB'] = True
    fw['UB_VALUE'] = fw.obj

    im['VALID_UB'] = True
    im['BEST_POSSIBLE_UB'] = (im.obj - im.obj_bound < 0.001)
    im['UB_VALUE'] = im.obj_bound

    return pd.concat([l, fw, im])

In [3]:
res = read_data()

In [4]:
bks = pd.read_csv('bks.csv')

In [5]:
def get_gap(row: pd.Series) -> float:
    i = row.instance_basename
    bks_row = bks[bks.instance_basename == i]

    if len(bks_row) == 0:
        print(f"No entry found in BKS file for instance {i}")
        raise RuntimeError('Missing BKS value')

    lb = bks_row.iloc[0]['BKS']
    ub = row['UB_VALUE']

    if row.VALID_UB and lb > ub + 0.1:
        print(f"Invalid bounds for {row.instance_basename}: {lb=:.2f}, {ub=:.2f}")
    elif row.VALID_UB and ub < lb < ub + 0.1:
        return 0

    if np.isnan(ub) or ub == 0:
        return 1
    else:
        return np.abs(ub - lb) / ub

In [6]:
res['GAP'] = res.apply(get_gap, axis=1)

In [7]:
def pct_valid_bound(df: pd.DataFrame) -> float:
    n_valid = len(df[df['VALID_UB'] == True])
    n_all = len(df)

    return 100 * n_valid / n_all

def pct_gap_valid_only(df: pd.DataFrame) -> float:
    valid = df[df['VALID_UB'] == True]
    return 100 * valid['GAP'].mean()

In [8]:
print('\\begin{tabular}{ll rr rr rr rrr rrr}')
print('\\toprule')
print(' & & ', end='')
print('\\multicolumn{2}{c}{\\textsc{UB-Lin}} & ', end='')
print('\\multicolumn{2}{c}{\\textsc{UB-PwLin}} & ', end='')
print('\\multicolumn{2}{c}{\\textsc{UB-Cont}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{UB-SSR}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{UB-SSR-2CE}} \\\\')
print('\\cmidrule(lr){3-4}\\cmidrule(lr){5-6}\\cmidrule(lr){7-8}\\cmidrule(lr){9-11}\\cmidrule(lr){12-14}')
print('\\(\\alpha^{\\text{Tsi}}\\) & \\(\\beta^{\\text{Tsi}}\\) & ', end='')
print('Gap\% & T (s) & Gap\% & T (s) & Gap\% & T (s) & ', end='')
print('Val\% & Gap\% & T (s) & Val\% & Gap\% & T (s) \\\\')
print('\\midrule')

for alpha in sorted(res.tsiligirides_hop_alpha.unique()):
    for beta in sorted(res.tsiligirides_hop_beta.unique()):
        f = res[(res.tsiligirides_hop_alpha == alpha) & (res.tsiligirides_hop_beta == beta)]

        print(f"{alpha:.2f} & {beta} & ", end='')

        # Linear UB
        df = f[f.algorithm == 'integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_LOOSE_with_vi1_with_vi4_liftMTZ']
        print(f"{pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

        # Piecewise-linear UB
        df = f[f.algorithm == 'integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_TIGHT_with_vi1_with_vi4_liftMTZ']
        print(f"{pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

        # Frank-Wolfe continuous relaxation UB
        df = f[f.algorithm == 'frankwolfe_with_vi1_with_vi4_liftMTZ']
        print(f"{pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

        # State-space relaxation labelling
        df = f[f.algorithm == 'labelling_RelaxedLabel']
        print(f"{pct_valid_bound(df):>5.2f} & {pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

        # State-space relaxation labelling with 2-cycle 
        df = f[f.algorithm == 'labelling_RelaxedTCLabel']
        print(f"{pct_valid_bound(df):>5.2f} & {pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} \\\\")

print('\\midrule')

print('\\multicolumn{2}{l}{Overall} & ', end='')

df = res[res.algorithm == 'integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_LOOSE_with_vi1_with_vi4_liftMTZ']
print(f"{pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

df = res[res.algorithm == 'integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_TIGHT_with_vi1_with_vi4_liftMTZ']
print(f"{pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

df = res[res.algorithm == 'frankwolfe_with_vi1_with_vi4_liftMTZ']
print(f"{pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

df = res[res.algorithm == 'labelling_RelaxedLabel']
print(f"{pct_valid_bound(df):>5.2f} & {pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} & ", end='')

df = res[res.algorithm == 'labelling_RelaxedTCLabel']
print(f"{pct_valid_bound(df):>5.2f} & {pct_gap_valid_only(df):>5.2f} & {df.time_s.mean():.1f} \\\\")

print('\\bottomrule')
print('\\end{tabular}')
        

\begin{tabular}{ll rr rr rr rrr rrr}
\toprule
 & & \multicolumn{2}{c}{\textsc{UB-Lin}} & \multicolumn{2}{c}{\textsc{UB-PwLin}} & \multicolumn{2}{c}{\textsc{UB-Cont}} & \multicolumn{3}{c}{\textsc{UB-SSR}} & \multicolumn{3}{c}{\textsc{UB-SSR-2CE}} \\
\cmidrule(lr){3-4}\cmidrule(lr){5-6}\cmidrule(lr){7-8}\cmidrule(lr){9-11}\cmidrule(lr){12-14}
\(\alpha^{\text{Tsi}}\) & \(\beta^{\text{Tsi}}\) & Gap\% & T (s) & Gap\% & T (s) & Gap\% & T (s) & Val\% & Gap\% & T (s) & Val\% & Gap\% & T (s) \\
\midrule
0.10 & 2 &  9.97 & 29.3 &  6.95 & 22.5 &  5.59 & 5.9 & 28.57 & 34.58 & 2742.3 & 20.41 & 13.81 & 3020.8 \\
0.10 & 3 & 16.29 & 33.2 & 11.70 & 30.3 &  8.23 & 5.4 & 28.57 & 29.83 & 2722.1 & 20.41 & 14.61 & 3035.8 \\
0.10 & 4 & 21.37 & 35.9 & 15.17 & 34.2 & 12.95 & 5.9 & 28.57 & 28.79 & 2772.4 & 18.37 & 10.63 & 3015.6 \\
0.10 & 5 & 23.31 & 36.7 & 17.41 & 35.8 & 12.96 & 6.2 & 28.57 & 31.86 & 2769.0 & 16.33 & 11.45 & 3046.9 \\
0.20 & 2 & 21.65 & 36.5 & 15.99 & 33.0 &  8.00 & 6.2 & 22.45 & 35.30 & 2893.

### Comparison of solution characteristics
#### Solutions from Baron vs. PwLin

In [10]:
pw = res[res.algorithm=='integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_TIGHT_with_vi1_with_vi4_liftMTZ']
br = pd.read_csv('baron-concave-bounds.csv')
br = br[br.algorithm == 'nonlinear_concave_model_with_vi1_liftMTZ']

In [13]:
pw.pct_custs_visited_which_are_tb.mean(), br.pct_custs_visited_which_are_tb.mean()

(27.46521309352883, 5.492142638938807)

In [17]:
1 - (pw.avg_travel_time_after_tb_cust / pw.tour_duration).mean(), 1 - (br.avg_travel_time_after_tb_cust / br.tour_duration).mean()

(0.7018914897186171, 0.9596684230753166)